In [1]:
import numpy as np
import tensorflow as tf

import os
import skimage.data
import skimage.transform

import matplotlib
import matplotlib.pyplot as plt

In [2]:
def load_data(data_dir):
    """Loads a data set and returns two lists:
    
    images: a list of Numpy arrays, each representing an image.
    labels: a list of numbers that represent the images labels.
    """
    # Get all subdirectories of data_dir. Each represents a label.
    directories = [d for d in os.listdir(data_dir) 
                   if os.path.isdir(os.path.join(data_dir, d))]
    # Loop through the label directories and collect the data in
    # two lists, labels and images.
    labels = []
    images = []
    for d in directories:
        label_dir = os.path.join(data_dir, d)
        file_names = [os.path.join(label_dir, f) 
                      for f in os.listdir(label_dir) if f.endswith(".ppm")]
        # For each label, load it's images and add them to the images list.
        # And add the label number (i.e. directory name) to the labels list.
        for f in file_names:
            images.append(skimage.data.imread(f))
            labels.append(int(d))
    return images, labels


# Load training and testing datasets.
ROOT_PATH = "/export/home/bzz5065/DOWNLOADS"
train_data_dir = os.path.join(ROOT_PATH, "Training")
test_data_dir = os.path.join(ROOT_PATH, "Testing")

images, labels = load_data(train_data_dir)



In [3]:
images32 = [skimage.transform.resize(image, (32, 32), mode='constant') for image in images]
# Prepare the training dataset
labels_a = np.array(labels)
images_a = np.array(images32)
print("labels: ", labels_a.shape, "\nimages: ", images_a.shape)

factor = 0.95
tr = int(len(labels)*factor)

X_dat = images_a
Y_dat = np.zeros((len(labels),62))
Y_dat[np.arange(len(labels)),labels] = 1
print("labels: ", Y_dat.shape, "\nimages: ", X_dat.shape)

X_train = X_dat#[:tr]
y_train = Y_dat#[:tr]
#X_valid = X_dat[tr:]
#y_valid = Y_dat[tr:]

print(y_train.shape)

labels:  (4575,) 
images:  (4575, 32, 32, 3)
labels:  (4575, 62) 
images:  (4575, 32, 32, 3)
(4575, 62)


In [4]:
# Prepare the test dataset.
test_images, test_labels = load_data(test_data_dir)

# Transform the images, just like we did with the training set.
test_images32 = [skimage.transform.resize(image, (32, 32), mode='constant') for image in test_images]

test_labels = np.array(test_labels)
test_images32 = np.array(test_images32)
print("labels: ", test_labels.shape, "\nimages: ", test_images32.shape)

X_test = test_images32
y_test = np.zeros((len(test_labels),62))
y_test[np.arange(len(test_labels)),test_labels] = 1
print("labels: ", y_test.shape, "\nimages: ", X_test.shape)

labels:  (2520,) 
images:  (2520, 32, 32, 3)
labels:  (2520, 62) 
images:  (2520, 32, 32, 3)


In [5]:
def model(x, logits=False, training=False):

    with tf.variable_scope('conv0'):
        z = tf.layers.conv2d(x, filters=32, kernel_size=[3, 3], padding='same', activation=tf.nn.relu)
        z = tf.layers.max_pooling2d(z, pool_size=[2, 2], strides=2)

    with tf.variable_scope('conv1'):
        z = tf.layers.conv2d(z, filters=64, kernel_size=[3, 3], padding='same', activation=tf.nn.relu)
        z = tf.layers.max_pooling2d(z, pool_size=[2, 2], strides=2)

    with tf.variable_scope('flat'):
        shape = z.get_shape().as_list()
        z = tf.reshape(z, [-1, np.prod(shape[1:])])

    with tf.variable_scope('dense'):
        z = tf.layers.dense(z, units=128, activation=tf.nn.relu)
        z = tf.layers.dropout(z, rate=0.25)

    logits_ = tf.layers.dense(z, units=62, name='logits')
    y = tf.nn.softmax(logits_, name='ybar')

    if logits == True: #if logits is True
        return y, logits_
    return y 

In [6]:
def jsma(model, x, y=None, epochs=1, eps=1.0, k=1, clip_min=0.0, clip_max=1.0, score_fn=lambda t, o: t * tf.abs(o)):

    n = tf.shape(x)[0] # how many images are input? n = 128 by batch size 

    target = tf.cond(tf.equal(0, tf.rank(y)), # tf.rank(y) returns rank of a tensor y
                     lambda: tf.zeros([n], dtype=tf.int32) + y,
                     lambda: y)

    target = tf.stack((tf.range(n), target), axis=1) # 2xn

    """
    x = tf.constant([1, 4])
    y = tf.constant([2, 5])
    z = tf.constant([3, 6])
    tf.stack([x, y, z], axis=1)  # [[1, 2, 3], [4, 5, 6]]
    """

    if isinstance(epochs, float):
        tmp = tf.to_float(tf.size(x[0])) * epochs
        epochs = tf.to_int32(tf.floor(tmp))

    return _jsma_impl(model, x, target, epochs=epochs, eps=eps, clip_min=clip_min, clip_max=clip_max, score_fn=score_fn)


def _prod(iterable):
    ret = 1
    for x in iterable:
        ret *= x
    return ret


def _jsma_impl(model, x, yind, epochs, eps, clip_min, clip_max, score_fn):

    def _cond(i, xadv):
        return tf.less(i, epochs)

    def _body(i, xadv):
        ybar = model(xadv)

        dy_dx = tf.gradients(ybar, xadv)[0] # Nx28x28x1


        # gradients of target w.r.t input
        yt = tf.gather_nd(ybar, yind) #yind = target = (2 x n) = random labels assigned by np.random function
        dt_dx = tf.gradients(yt, xadv)[0]   #[0] makes it list then you can do do_dx = dy_dx - dt_dx # Nx28x28x1

        # gradients of non-targets w.r.t input
        do_dx = dy_dx - dt_dx # Nx28x28x1

        c0 = tf.logical_or(eps < 0, xadv < clip_max) # returns true when either of these two is true
        c1 = tf.logical_or(eps > 0, xadv > clip_min)
        cond = tf.reduce_all([dt_dx >= 0, do_dx <= 0, c0, c1], axis=0)
        cond = tf.to_float(cond) #tf.to_float([1,2,3]) produces just [1.,2.,3.] # return 0 or return 1

        # saliency score for each pixel
        score = cond * score_fn(dt_dx, do_dx) # function to calculate the saliency score for each pixel

        shape = score.get_shape().as_list() # 784 x 1 matrix # To get the shape as a list of ints, do tensor.get_shape().as_list()
        dim = _prod(shape[1:]) # multiplication except the first one
        score = tf.reshape(score, [-1, dim]) # make 1D matrix


        # find the pixel with the highest saliency score
        ind = tf.argmax(score, axis=1) # find the pixel with highest value
        dx = tf.one_hot(ind, dim, on_value=eps, off_value=0.0)
        dx = tf.reshape(dx, [-1] + shape[1:])

        xadv = tf.stop_gradient(xadv + dx)
        xadv = tf.clip_by_value(xadv, clip_min, clip_max)

        return i+1, xadv

    _, xadv = tf.while_loop(_cond, _body, (0, tf.identity(x)), back_prop=False, name='_jsma_batch')

    return xadv


In [7]:
#tf.reset_default_graph() 

class Environment():
    pass

env = Environment()

with tf.variable_scope('model', reuse=False):
    env.x = tf.placeholder(tf.float32, (None, 32, 32, 3))
    env.y = tf.placeholder(tf.float32, (None, 62), name='y')

    env.ybar, logits = model(env.x, logits=True)

    with tf.variable_scope('acc'):
        count = tf.equal(tf.argmax(env.y, axis=1), tf.argmax(env.ybar, axis=1))
        env.acc = tf.reduce_mean(tf.cast(count, tf.float32), name='acc')

    with tf.variable_scope('loss'):
        cross_entropy = tf.nn.softmax_cross_entropy_with_logits(labels=env.y, logits=logits)
        env.loss = tf.reduce_mean(cross_entropy, name='loss')

    with tf.variable_scope('train_op'):
        optimizer = tf.train.AdamOptimizer()
        env.train_op = optimizer.minimize(env.loss)

with tf.variable_scope('model', reuse=True):
    env.target = tf.placeholder(tf.int32, (), name='target')
    env.adv_epochs = tf.placeholder_with_default(20, shape=(), name='epochs')
    env.adv_eps = tf.placeholder_with_default(0.2, shape=(), name='eps')
    env.x_jsma = jsma(model, env.x, env.target, eps=env.adv_eps, epochs=env.adv_epochs)

In [8]:
def training(sess, env, X_data, Y_data, shuffle=True, batch=32, epochs=1):
	Xshape = X_data.shape
	n_data = Xshape[0]
	n_batches = int(n_data/batch)

	print(X_data.shape)


	for ep in range(epochs):
		print('epoch number: ', ep+1)
		if shuffle:
			ind = np.arange(n_data)
			np.random.shuffle(ind)
			X_data = X_data[ind]
			Y_data = Y_data[ind]

		for i in range(n_batches):
			print(' batch {0}/{1}'.format(i + 1, n_batches), end='\r')
			start = i*batch 
			end = min(start+batch, n_data)

			sess.run(env.train_op, feed_dict={env.x: X_data[start:end], env.y: Y_data[start:end]})

		evaluate(sess, env, X_test, y_test)
		
        

In [9]:
def evaluate(sess, env, X_test, Y_test, batch=32):
	n_data = X_test.shape[0]
	n_batches = int(n_data/batch)

	totalAcc = 0
	totalLoss = 0


	for i in range(n_batches):
		#print('batch ', i)
		print(' batch {0}/{1}'.format(i + 1, n_batches), end='\r')
		start = i*batch 
		end = min(start+batch, n_data)
		batch_X = X_test[start:end]
		batch_Y = Y_test[start:end]

		batch_loss, batch_acc = sess.run([env.loss, env.acc], feed_dict={env.x: batch_X, env.y: batch_Y})

		totalAcc = totalAcc + batch_acc*(end-start)
		totalLoss = totalLoss + batch_loss*(end-start)
		

	totalAcc = totalAcc/n_data
	totalLoss = totalLoss/n_data
	print('acc: {0:.3f} loss: {1:.3f}'.format(totalAcc, totalLoss))
	return totalAcc, totalLoss

In [14]:
def make_jsma(sess, env, X_data, epochs=0.2, eps=1.0, batch_size=128):
    """
    Generate JSMA by running env.x_jsma.
    """
    print('\nMaking adversarials via JSMA')

    n_sample = X_data.shape[0]
    n_batch = int((n_sample + batch_size - 1) / batch_size)
    X_adv = np.empty_like(X_data)

    for batch in range(n_batch):
        print(' batch {0}/{1}'.format(batch + 1, n_batch), end='\r')
        start = batch * batch_size
        end = min(n_sample, start + batch_size)
        feed_dict = {
            env.x: X_data[start:end],
            env.target: np.random.choice(n_classes),
            env.adv_epochs: epochs,
            env.adv_eps: eps}
        adv = sess.run(env.x_jsma, feed_dict=feed_dict)
        X_adv[start:end] = adv
    print()

    return X_adv

n_classes = 10

In [ ]:
sess = tf.InteractiveSession()
sess.run(tf.global_variables_initializer())
sess.run(tf.local_variables_initializer())

print('\nTraining')

training(sess, env, X_train, y_train, epochs=5)

print('\nEvaluating on clean data')

evaluate(sess, env, X_test, y_test)

print('\nGenerating adversarial data')

X_adv = make_jsma(sess, env, X_test, epochs=80, eps=1)

print('\nEvaluating on adversarial data')

evaluate(sess, env, X_adv, y_test)


	


Training
(4575, 32, 32, 3)
epoch number:  1
